In [1]:
import pandas as pd

In [2]:
metadata_df = pd.read_csv(r"C:\Users\garla\first-node\WalkExplorer\open_ai\Adjusted_CLIP_and_OpenAI_Scores.csv")
updated_edges_df = pd.read_csv(r"C:\Users\garla\first-node\WalkExplorer\routing\data\new_updated_edges_2.csv")
scores_openai_clip_df = pd.read_csv(r"C:\Users\garla\first-node\WalkExplorer\open_ai\OpenAI_and_CLIP_Scores.csv")

image_col = ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10',
             'I11', 'I12', 'I13', 'I14', 'I15', 'I16', 'I17', 'I18', 'I19', 'I20',
             'I21', 'I22', 'I23', 'I24', 'I25', 'I26', 'I27', 'I28', 'I29']

for i in image_col:
    updated_edges_df[i] = updated_edges_df[i].replace("Not found", "")

# Iterate through each row and store image names in a list
# Edges with no image have an empty list
# updated_edges_df['image_list'] = updated_edges_df[image_col].apply(lambda row: [i for i in row if i != ""], axis=1)

print(updated_edges_df['image_list'].iloc[0])

C:\Users\garla\AppData\Local\Temp\ipykernel_22128\3064901798.py:2: DtypeWarning: Columns (16,18,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  updated_edges_df = pd.read_csv(r"C:\Users\garla\first-node\WalkExplorer\routing\data\new_updated_edges_2.csv")


['image_19104', 'image_7023']


In [95]:
metadata_df.set_index('ImageID', inplace=True)
scores_openai_clip_df.set_index('ImageID', inplace=True)


In [132]:

def calculate_CLIP_score(image_list):
    if not image_list:
        return 0
    # Retrieve scores for images in the list
    scores = [metadata_df.loc[image, 'Adjusted CLIP'] for image in image_list if image in metadata_df.index]
    if not scores:
        return 0
    # Return the average score
    return sum(scores) / len(scores)

def calculate_OPENAI_score(image_list):
    if not image_list:
        return 0
    # Retrieve scores for images in the list
    scores = [metadata_df.loc[image, 'OpenAI Score'] for image in image_list if image in metadata_df.index]
    if not scores:
        return 0
    # Return the average score
    return sum(scores) / len(scores)

def calculate_PROMPT_score(image_list , prompt):
    if not image_list:
        return 0
    # Retrieve scores for images in the list
    scores = [scores_openai_clip_df.loc[image, prompt] for image in image_list if image in scores_openai_clip_df.index]
    if not scores:
        return 0
    # Return the average score
    return sum(scores) / len(scores)

In [133]:
# All edge scores are the average of all the scores from 'image_list'
# Calculate the CLIP Score
updated_edges_df['CLIP Score'] = updated_edges_df['image_list'].apply(calculate_CLIP_score)

# Calculate the OpenAI Score
updated_edges_df['OPENAI Score'] = updated_edges_df['image_list'].apply(calculate_OPENAI_score)


print(updated_edges_df['OPENAI Score'].iloc[0])
print(updated_edges_df['CLIP Score'].iloc[0])


0
0


In [150]:

# Filter out rows where the CLIP Score or OPENAI Score is 0
updated_edges_df['CLIP Score'] = pd.to_numeric(updated_edges_df['CLIP Score'], errors='coerce')
updated_edges_df['OPENAI Score'] = pd.to_numeric(updated_edges_df['OPENAI Score'], errors='coerce')
filtered_df = updated_edges_df[(updated_edges_df['CLIP Score'] != 0) & (updated_edges_df['OPENAI Score'] != 0)]

# Calculate mean, median, mode, standard deviation, 25% and 75% quartiles for CLIP Score
clip_count = filtered_df['CLIP Score'].count()
clip_min = filtered_df['CLIP Score'].min()
clip_max = filtered_df['CLIP Score'].max()
clip_mean = filtered_df['CLIP Score'].mean()
clip_median = filtered_df['CLIP Score'].median()
clip_mode = filtered_df['CLIP Score'].mode()[0]  # mode() returns a Series; take the first mode
clip_std = filtered_df['CLIP Score'].std()
clip_25_quartile = filtered_df['CLIP Score'].quantile(0.25)
clip_75_quartile = filtered_df['CLIP Score'].quantile(0.75)

# Calculate mean, median, mode, standard deviation, 25% and 75% quartiles for OPENAI Score
openai_count = filtered_df['OPENAI Score'].count()
openai_min = filtered_df['OPENAI Score'].min()
openai_max = filtered_df['OPENAI Score'].max()
openai_mean = filtered_df['OPENAI Score'].mean()
openai_median = filtered_df['OPENAI Score'].median()
openai_mode = filtered_df['OPENAI Score'].mode()[0]  # mode() returns a Series; take the first mode
openai_std = filtered_df['OPENAI Score'].std()
openai_25_quartile = filtered_df['OPENAI Score'].quantile(0.25)
openai_75_quartile = filtered_df['OPENAI Score'].quantile(0.75)


# Print the results
print("CLIP Score Analysis:")
print(f"Count: {clip_count}")
print(f"Min: {clip_min}")
print(f"Max: {clip_max}")
print(f"Mean: {clip_mean}")
print(f"Median: {clip_median}")
print(f"Mode: {clip_mode}")
print(f"Standard Deviation: {clip_std}")
print(f"25% Quartile: {clip_25_quartile}")
print(f"75% Quartile: {clip_75_quartile}")

print("\nOPENAI Score Analysis:")
print(f"Count: {openai_count}")
print(f"Min: {openai_min}")
print(f"Max: {openai_max}")
print(f"Mean: {openai_mean}")
print(f"Median: {openai_median}")
print(f"Mode: {openai_mode}")
print(f"Standard Deviation: {openai_std}")
print(f"25% Quartile: {openai_25_quartile}")
print(f"75% Quartile: {openai_75_quartile}")


CLIP Score Analysis:
Count: 37724
Min: 1.9290299197350824
Max: 9.408783561485976
Mean: 5.973495327412755
Median: 5.973495327412756
Mode: 5.973495327412756
Standard Deviation: 0.8580697680820101
25% Quartile: 5.483071951072347
75% Quartile: 6.510211790926131

OPENAI Score Analysis:
Count: 37724
Mean: 1.0
Median: 10.0
Mean: 5.339487447988676
Median: 5.339487447988676
Mode: 8.0
Standard Deviation: 1.9775702924535106
25% Quartile: 4.0
75% Quartile: 7.0


In [138]:
# Number of edges without images
clip_zero_count = (updated_edges_df['CLIP Score'] == 0).sum()
print(f"Number of edges w/o images: {clip_zero_count}")

# Replace zeroes with the respective averages
updated_edges_df['CLIP Score'] = updated_edges_df['CLIP Score'].replace(0, clip_mean)
updated_edges_df['OPENAI Score'] = updated_edges_df['OPENAI Score'].replace(0, openai_mean)


Number of edges w/o images: 2156


In [164]:
# Add a new column for the difference between CLIP Score and OPENAI Score
updated_edges_df['score_difference'] = updated_edges_df['CLIP Score'] - updated_edges_df['OPENAI Score']

# Calculate mean, median, mode, standard deviation, 25% and 75% quartiles for OPENAI Score
score_diff_count = updated_edges_df['score_difference'].count()
score_diff_min = updated_edges_df['score_difference'].min()
score_diff_max = updated_edges_df['score_difference'].max()
score_diff_mean = updated_edges_df['score_difference'].mean()
score_diff_median = updated_edges_df['score_difference'].median()
score_diff_mode = updated_edges_df['score_difference'].mode()[0]
score_diff_std = updated_edges_df['score_difference'].std()
score_diff_25_quartile = updated_edges_df['score_difference'].quantile(0.25)
score_diff_75_quartile = updated_edges_df['score_difference'].quantile(0.75)

# Print the results
print("Score Difference(CLIP - OpenAI) Analysis:")
print(f"Count: {score_diff_count}")
print(f"Min: {score_diff_min}")
print(f"Max: {score_diff_max}")
print(f"Mean: {score_diff_mean}")
print(f"Median: {score_diff_median}")
print(f"Mode: {score_diff_mode}")
print(f"Standard Deviation: {score_diff_std}")
print(f"25% Quartile: {score_diff_25_quartile}")
print(f"75% Quartile: {score_diff_75_quartile}")


Score Difference(CLIP - OpenAI) Analysis:
Count: 37724
Min: -5.596365623877022
Max: 8.310254471945786
Mean: 0.6340078794240798
Median: 0.6340078794240807
Mode: 0.6340078794240807
Standard Deviation: 1.7891225428521604
25% Quartile: -0.6979402888423394
75% Quartile: 1.8267921670714493


In [90]:
# The adjusted_score of the CLIP score
# Gives lowest score to the highest CLIP score, which is used in Dijkstra's algorithm and as a weight in path construction
updated_edges_df = pd.read_csv(r"C:\Users\garla\first-node\WalkExplorer\routing\data\new_updated_edges_2.csv")

updated_edges_df["adjusted_score"] = 10 - updated_edges_df['CLIP Score']
updated_edges_df["adjusted_OPENAI_score"] = 11 - updated_edges_df['CLIP Score']


In [100]:
# Calculate the prompt scores. Take the average from 'image_list'
prompts = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10',
             'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20',
             'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30',
             'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39', 'P40',
             'P41', 'P42', 'P43', 'P44', 'P45', 'P46', 'P47', 'P48', 'P49', 'P50']

prompt_description = [
    'Paved sidewalk', 'No sidewalk', 'Continous sidewalk', 'Incomplete sidewalk', 'Smooth sidewalk',
    'Cracked sidewalk', 'Wide sidewalk', 'Narrow sidewalk', 'Buffer zone between sidewalk and street', 
    'No buffer zone', 'No construction', 'Construction on sidewalk', 'No tents', 'Tents on sidewalk', 
    'No scaffolding', 'Scaffolding', 'Streetlights', 'No streetlights', 'Clear intersection', 
    'Busy intersection', 'Pedestrian crossing signal', 'No pedestrian crossing signal', 'Pedestrian crosswalk',
    'No pedestrian crosswalk', 'Traffic light', 'No traffic light', 'Stop sign', 'No stop sign', 
    'One car lane', 'Many car lanes', 'No railroad tracks', 'Railroad tracks', 'No stairs', 'Stairs',
    'Outdoor seating', 'No outdoor seating', 'Public transportation stop', 'No public transportation', 
    'No trash or dumpsters', 'Trash or dumpsters', 'No graffiti', 'Graffiti', 'Populated area', 
    'Abandoned building', 'Trees, grass, or flowers', 'No trees, grass or flowers', 
    'Recreational park or playground', 'No recreational park or playground', 'Shops or restaurants', 
    'No shops or restaurants'
]

# Create a dictionary pairing prompts (P1 to P50) with the descriptions
prompt_dict = {f'P{i+1}': prompt_description[i] for i in range(len(prompt_description))}

# Print the dictionary
# print(prompt_dict)

for prompt, description in prompt_dict.items():
    updated_edges_df[prompt] = updated_edges_df['image_list'].apply(lambda x: calculate_PROMPT_score(x, description))
    
updated_edges_df.head()

,u,v,key,osmid,highway,oneway,reversed,length,lanes,name,...,P41,P42,P43,P44,P45,P46,P47,P48,P49,P50
0,29938143,321336630,0,29214061,footway,False,FALSE,13.325,NaN,NaN,...,20.448514,17.386956,21.427862,17.600812,18.965015,21.813179,23.285967,22.586676,19.843121,21.454141
1,30079057,10653494673,0,22927964,primary,False,FALSE,7.305,4,Stewart Street,...,19.979859,17.786411,21.09474,20.109436,19.9024,21.924886,23.537389,23.57215,22.373323,21.750458
2,30079057,10653494674,0,169473535,unclassified,False,FALSE,6.057,1,John Street,...,19.720381,17.558366,21.133794,21.081015,20.744279,21.655738,23.10246,22.908377,21.833847,20.826135
3,30079057,10653494670,0,428237363,secondary,False,TRUE,5.404,4,Eastlake Avenue East,...,20.174785,17.881676,21.621544,20.094789,21.033147,22.300113,23.758795,23.431641,22.150952,21.563752
4,30079057,10653494669,0,"[623957704, 393047812]",secondary,False,TRUE,14.680,2,Eastlake Avenue East,...,20.209366,18.155878,21.485521,19.652466,18.749135,21.671901,22.019949,22.836645,22.359774,21.994142


In [141]:
prompts = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10',
             'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20',
             'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30',
             'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'P38', 'P39', 'P40',
             'P41', 'P42', 'P43', 'P44', 'P45', 'P46', 'P47', 'P48', 'P49', 'P50']
# Loop through each prompt and replace zeroes with the average
for prompt in prompts:
    # convert prompt score to numeric data type
    updated_edges_df[prompt] = pd.to_numeric(updated_edges_df[prompt], errors='coerce')
    
    # Calculate the average of the current prompt, excluding zeroes
    avg = updated_edges_df.loc[updated_edges_df[prompt] != 0, prompt].mean()
    
    # Replace zeroes in the current prompt with the calculated average
    updated_edges_df[prompt] = updated_edges_df[prompt].replace(0, avg)
    
    # Print confirmation for each prompt
    print(f"Zeroes in {prompt} have been replaced with the average: {avg}")

Zeroes in P1 have been replaced with the average: 22.41660458444656
Zeroes in P2 have been replaced with the average: 22.91473528598657
Zeroes in P3 have been replaced with the average: 21.941047209892115
Zeroes in P4 have been replaced with the average: 22.05294037545724
Zeroes in P5 have been replaced with the average: 20.91483374846554
Zeroes in P6 have been replaced with the average: 17.81306310979524
Zeroes in P7 have been replaced with the average: 23.010202213406057
Zeroes in P8 have been replaced with the average: 22.626104893306472
Zeroes in P9 have been replaced with the average: 26.000920068736672
Zeroes in P10 have been replaced with the average: 21.712381703041714
Zeroes in P11 have been replaced with the average: 20.037908402030197
Zeroes in P12 have been replaced with the average: 21.143587987693962
Zeroes in P13 have been replaced with the average: 20.87966807786881
Zeroes in P14 have been replaced with the average: 21.315533161528656
Zeroes in P15 have been replaced wi

In [3]:
# Change 'image_list' from string to dictionary
import ast

# Define a function to convert the string to a dictionary
def convert_to_dict(image_list_str):
    # Evaluate the string to a list using ast.literal_eval
    image_list = ast.literal_eval(image_list_str)
    
    # Convert the list to a dictionary where keys are indices and values are the image names
    return {str(i): image for i, image in enumerate(image_list)}

# Apply the function to the image_list column
updated_edges_df['image_list'] = updated_edges_df['image_list'].apply(convert_to_dict)

# Check the first few rows to see the result
print(updated_edges_df['image_list'])

0                  {'0': 'image_19104', '1': 'image_7023'}
1                   {'0': 'image_358', '1': 'image_26290'}
2                    {'0': 'image_358', '1': 'image_2837'}
3                   {'0': 'image_358', '1': 'image_24785'}
4        {'0': 'image_358', '1': 'image_5223', '2': 'im...
                               ...                        
37791                                 {'0': 'image_16237'}
37792                                                   {}
37793                                                   {}
37794                                                   {}
37795                                                   {}
Name: image_list, Length: 37796, dtype: object


In [4]:
updated_edges_df.to_csv(r"C:\Users\garla\first-node\WalkExplorer\routing\data\new_updated_edges_2.csv", index = False)